In [1]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
from scipy import spatial
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt
%matplotlib inline

git_path = '/Users/optas/Documents/Git_Repos/'
sys.path.insert(0, git_path)
import autopredictors.tf_wrapper.basics as basics
from autopredictors import *

In [2]:
from tensorflow.models.image.cifar10 import cifar10
cifar10.FLAGS.data_dir = '/Users/optas/DATA/CIFAR_10/'
cifar10.maybe_download_and_extract()
NUM_CLASSES = 10

In [3]:
images, labels = cifar10.distorted_inputs()
paddings = [[0,0], [50, 50,], [50, 50], [0,0]]
images_big = tf.pad(images, paddings, "CONSTANT")


Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [4]:
# g = tf.get_default_graph()

def test_vgg_m_conv(in_signal, keep_prob, g, stddev=5e-2):
    '''
    DOI: Return of the Devil in the Details: Delving Deep into Convolutional Nets: Ken Chatfield et al.    
    '''    
    with tf.variable_scope('test_vgg') as scope:

#         scope.reuse_variables()
                
        conv = convolutional_layer

        layer = conv(in_signal, n_filters=96, filter_size=[7,7], stride=2, padding='SAME', stddev=stddev, name="conv_1")
        layer = max_pool(relu(layer), ksize=(3,3), stride=(2,2), name='max_pool_1')

        layer = conv(layer, n_filters=256, filter_size=[5,5], stride=2, padding='SAME', stddev=stddev, name="conv_2")
        layer = max_pool(relu(layer), ksize=(3,3), stride=(2,2), name='max_pool_2')

        layer = conv(layer, n_filters=512, filter_size=[3,3], stride=1, padding='SAME', stddev=stddev, name="conv_3")
        layer = relu(layer)

        layer = conv(layer, n_filters=512, filter_size=[3,3], stride=1, padding='SAME', stddev=stddev, name="conv_4")
        layer = relu(layer)

        layer = conv(layer, n_filters=512, filter_size=[3,3], stride=1, padding='SAME', stddev=stddev, name="conv_5")
        layer = max_pool(relu(layer), ksize=(3,3), stride=(2,2), name='max_pool_3')

        out_dim = 30
    
        fc = fully_connected_layer(layer, out_dim, stddev, init_bias=0, name='fc')        
        fc_w = g.get_tensor_by_name('test_vgg/fc/weights:0')
        fc_cv = basics.fully_conected_via_convolutions(layer, out_dim, fc_w, init_bias=0, name='convolutional_fc')
        
    return fc_cv, fc
    
    


In [5]:
with tf.Graph().as_default() as g:
    
        # Get images and labels for CIFAR-10.
    images, labels = cifar10.distorted_inputs()
    fc_cv, fc = test_vgg_m_conv(images, 1, g)
    init = tf.initialize_all_variables()    
    
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(init)
#     # Start the queue runners.
    tf.train.start_queue_runners(sess=sess)
    
    a,b= sess.run([fc_cv, fc])
    

    
    



Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [7]:
print a[2,:]
print b[2,:]

[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
